In [121]:
import torch 
from torchvision import models
from torch import nn
from torchvision import transforms
from PIL import Image
import numpy as np

In [122]:
res = models.resnet18(pretrained=True)
# res.fc = nn.Linear(512,512)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print(res)
res.to("cuda")

In [ ]:
len(list(res.children()))
res

In [133]:
  modules = list(res.children())[:-1]
  res = torch.nn.Sequential(*modules)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
!unzip /content/drive/MyDrive/AIP/Train_Test_dataset.zip

In [126]:
def preprocess_image(img_path):

    preprocess = transforms.Compose([
        transforms.Resize((224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])])
    
    img = Image.open(img_path)
    if img.mode != 'RGB':
      img = img.convert('RGB')
    img = preprocess(img).unsqueeze(0)
    return img

In [134]:
def get_output(img_path):
  img = preprocess_image(img_path)
  img = img.cuda()
  features = res(img)[0,:,0,0]
  return features

In [129]:
import numpy as np
import matplotlib.pyplot as plt
import glob

# Training Data set

In [135]:
bear = []
butterfly = []
camel = []
chimp = []
duck = []
elephant = []

res.eval()

for img_path in glob.glob("/content/classification_dataset/train/bear/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),0)
  bear.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/train/butterfly/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),1)
  butterfly.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/train/camel/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),2)
  camel.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/train/chimp/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),3)
  chimp.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/train/duck/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),4)
  duck.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/train/elephant/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),5)
  elephant.append(arr_0)

In [136]:
X_train = np.vstack((bear,butterfly,camel,chimp,duck,elephant))
np.save("Train_Data_Points", X_train) 
len(X_train[0])

513

# Testing Data set

In [137]:
bear = []
butterfly = []
camel = []
chimp = []
duck = []
elephant = []

res.eval()

for img_path in glob.glob("/content/classification_dataset/test/bear/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),0)
  bear.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/test/butterfly/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),1)
  butterfly.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/test/camel/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),2)
  camel.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/test/chimp/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),3)
  chimp.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/test/duck/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),4)
  duck.append(arr_0)

for img_path in glob.glob("/content/classification_dataset/test/elephant/*.jpg"):
  arr_0 = np.append(get_output(img_path).cpu().detach().numpy(),5)
  elephant.append(arr_0)

In [138]:
X_test = np.vstack((bear,butterfly,camel,chimp,duck,elephant))
np.save("Test_Data_Points", X_test) 
len(X_test[0])

513

#Loading data and performing KNN for accuracy checking


In [139]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [140]:
data_train = np.load("Train_Data_Points.npy")
data_test = np.load("Test_Data_Points.npy")

np.random.shuffle(data_train)
np.random.shuffle(data_test)

In [141]:
X_train = data_train[:,:512]
y_train = data_train[ : ,512]

X_test = data_test[:,:512]
y_test = data_test[ : ,512]

In [164]:
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [170]:
# create the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# calculate the accuracy
accuracy = (conf_matrix[0,0] + conf_matrix[1,1] + conf_matrix[2,2] + conf_matrix[3,3] + conf_matrix[4,4] + conf_matrix[5,5]) / conf_matrix.sum()

print("Accuracy:", accuracy)

Accuracy: 0.95
